<a href="https://colab.research.google.com/github/Peterpal1/CyborgDroids/blob/main/TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>